# Preparing foreground & background masks

## Includes

In [ ]:
# mass includes
import os, sys, warnings
import torch as t
from sklearn import mixture
from PIL import Image
from tqdm.notebook import tqdm

# add paths for all sub-folders
paths = [root for root, _, _ in os.walk('.')\
         if 'evals' not in root]
for item in paths:
    sys.path.append(item)

from ipynb.fs.full.config import Config
from ipynb.fs.full.dataLoader import *

## Initialization

In [ ]:
# load config
opt = Config()

# dataset for training
train_dataset = ImageSet(opt,
                         mode='all',
                         norm=False,
                         rand_trans=False,
                         mask_out=False)
train_loader = t.utils.data.DataLoader(train_dataset)

## Generate masks

In [ ]:
# loop over all images
for img, f_name in tqdm(train_loader, desc='progress',
                        total=len(train_loader)):
    img = img.squeeze()

    # fit binary GMM and predict labels
    in_data = img.permute(0, 2, 3, 1).reshape(-1, 3).numpy()
    gmm = mixture.GaussianMixture(n_components=2,
                                  covariance_type='full',
                                  init_params='k-means++',
                                  max_iter=1000,
                                  random_state=42).fit(in_data)

    _, _, h, w = img.size()
    labels = gmm.predict(in_data)

    # generate binary mask (assume dominant class is the background, experimental)
    means = gmm.means_.sum(-1)
    if np.sum(1 - labels) > np.sum(labels):
        mask = labels.reshape(4, h, w).astype('bool')
    else:
        mask = (1 - labels.reshape(4, h, w)).astype('bool')

    # save mask to image
    for f_idx in range(4):
        file_path = os.path.join(opt.data_path, f_name[f_idx][0] + '-mask.png')
        mask_img = Image.fromarray(mask[f_idx, :, :])
        mask_img.save(file_path, bits=1, optimize=True)